In [5]:
import xmltodict
import pandas as pd
import os
import re

# Cleaning

In [6]:
def remove_tags(read_file):
    read_file = re.sub(b'</?external-xref[^<>]*>',b'', read_file)
    read_file = re.sub(b'<quote>',b'"', read_file)
    read_file = re.sub(b'</quote>',b'"', read_file)
    read_file = re.sub(b'</?term[^<>]*>',b' ', read_file)
    read_file = re.sub(b'</?pagebreak[^<>]*>',b'', read_file)
    return read_file

In [7]:
def flatten(d):
    output = ''
    for k, v in d.items():
        if not k.startswith("@"):
            if isinstance(v, dict):
                output += flatten(v)
            elif isinstance(v, list):
                for l in v:
                    if isinstance(l, dict):
                        output += flatten(l)
                    else:
                        if l:
                            output += l + ' '
            else:
                if v:
                    output += v + ' '
    return output

In [8]:
def bill_to_dict(filename, doc):
    def _clean_body(bodydict):
        clean_bodydict = {}
        for k, v in bodydict.items():
            if not k.startswith("@"):
                if isinstance(v, list):
                    sec_str = ''
                    for sub_dict in v:
                        if sub_dict:
                            sec_str += flatten(sub_dict)
                    clean_bodydict[k] = sec_str
#                 elif isinstance(v, dict):
                else:
                    clean_bodydict[k] = flatten(v)
#                 else:
#                     print('xxxxxxxxx')
        return clean_bodydict
    
    bill_type = list(doc)[0]
    data_dict = {}
    
    data_dict["file-name"] = filename
    data_dict["bill-type"] = bill_type
    
    metadata = {}
    for n in list(doc[bill_type]):
        # group meta data
        if n.startswith("@"): 
            metadata[n] = doc[bill_type][n]
        # unify name for different types
        elif n in ['legis-body', 'resolution-body', 'engrossed-amendment-body']:
            if isinstance(doc[bill_type][n], dict):
                data_dict['body'] = _clean_body(doc[bill_type][n])
            elif isinstance(doc[bill_type][n], list):
                # just take last one
                data_dict['body'] = _clean_body(doc[bill_type][n][-1])
            else:
                print('NOT dict nor list')
                data_dict['body'] = doc[bill_type][n]
                
        elif n == 'engrossed-amendment-form':
            data_dict['form'] = doc[bill_type][n]
        else:
            data_dict[n] = doc[bill_type][n]
    data_dict["metadata"] = metadata
    return data_dict

In [9]:
# I put all the files in one folder for simplicity
indir = 'bill_text_115' 

data = [] # a list of dict, each element is a xml file

for root, dirs, filenames in os.walk(indir):
    for filename in filenames:
        with open(os.path.join(indir, filename), 'rb') as f:
            doc = xmltodict.parse(remove_tags(f.read()))
            data.append(bill_to_dict(filename, doc))

In [ ]:
# [list(x) for x in set(tuple(x) for x in data)]

# Select Some Columns

In [10]:
select_keys = ['metadata', 'bill-type', 'body', 'file-name']
select_data = []
for d in data:
    select_data.append({ select_key: d[select_key] for select_key in select_keys })

In [11]:
df = pd.io.json.json_normalize(select_data)

In [13]:
df.head()

,bill-type,body.amendment,body.division,body.section,body.title,file-name,metadata.@amend-degree,metadata.@amend-stage,metadata.@amend-type,metadata.@bill-stage,metadata.@bill-type,metadata.@dms-id,metadata.@key,metadata.@public-print,metadata.@public-private,metadata.@resolution-stage,metadata.@resolution-type,metadata.@stage-count,metadata.@star-print
0,bill,NaN,NaN,1. Short title National Criminal Justice Commi...,NaN,US_Bill_Text_115_HR1607_IH.xml,NaN,NaN,NaN,Introduced-in-House,olc,H2015A7070C3246099BCA067076E3EACF,H,NaN,public,NaN,NaN,NaN,NaN
1,bill,NaN,NaN,1. Short title Supporting Families in Substanc...,NaN,US_Bill_Text_115_HR2857_EH.xml,NaN,NaN,NaN,Engrossed-in-House,olc,H0F11EF81EB9145A182D41E7ACF6E4F8E,H,NaN,public,NaN,NaN,1,NaN
2,bill,NaN,NaN,1. Short title State Cyber Resiliency Act This...,NaN,US_Bill_Text_115_S516_IS.xml,NaN,NaN,NaN,Introduced-in-Senate,olc,A1,NaN,NaN,public,NaN,NaN,NaN,NaN
3,bill,NaN,NaN,1. Short title Bureau of Reclamation Transpare...,NaN,US_Bill_Text_115_S216_IS.xml,NaN,NaN,NaN,Introduced-in-Senate,olc,A1,NaN,NaN,public,NaN,NaN,NaN,NaN
4,bill,NaN,NaN,1. Short title Synthetic Drug Awareness Act of...,NaN,US_Bill_Text_115_HR449_IH.xml,NaN,NaN,NaN,Introduced-in-House,olc,HCFB79B8702D342E4BB5C604694A3EA12,H,NaN,public,NaN,NaN,NaN,NaN


In [ ]:
df.to_pickle("data/bills.pkl") 

# Simple stats

In [16]:
df['bill-type'].value_counts()

bill             6712
resolution       1312
amendment-doc      15
Name: bill-type, dtype: int64